<a href="https://colab.research.google.com/github/cheminfoiict-boop/Godavari-BOB/blob/main/Cauvery_Comparison_Godavari_Workflow_2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import ee

PROJECT_ID = 'project-b75af78d-ff46-4d69-ab3'  # your project ID
ee.Authenticate()  # only if token expired
ee.Initialize(project=PROJECT_ID)

print("Earth Engine ready.")

In [ ]:
# Krishna Delta AOI (covers main delta front, Diviseema, Machilipatnam, Kolleru)
krishna_aoi = ee.Geometry.Rectangle([80.5, 15.6, 81.5, 16.6])

def add_ndwi(image):
    ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')
    return image.addBands(ndwi)

# Older: 1990–2010 Landsat 5/7 median
ls_old_k = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')) \
    .filterBounds(krishna_aoi) \
    .filterDate('1990-01-01', '2010-12-31') \
    .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
    .map(add_ndwi) \
    .select('NDWI') \
    .median().clip(krishna_aoi)

# Recent: 2015–2025 Landsat 8/9 median
ls_recent_k = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .merge(ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')) \
    .filterBounds(krishna_aoi) \
    .filterDate('2015-01-01', '2025-12-31') \
    .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
    .map(add_ndwi) \
    .select('NDWI') \
    .median().clip(krishna_aoi)

# Change: older - recent (negative = loss/erosion)
ndwi_change_krishna = ls_old_k.subtract(ls_recent_k).rename('NDWI_Change')

# Export
task_krishna = ee.batch.Export.image.toDrive(
    image=ndwi_change_krishna,
    description='Krishna_NDWI_Change_1990-2025',
    folder='Godavari_Figures',
    scale=30,
    region=krishna_aoi,
    maxPixels=1e10
)
task_krishna.start()

print("Krishna NDWI export started.")
print("Task ID:", task_krishna.id)
print("Check status: https://code.earthengine.google.com/tasks")
print("Output file: Drive → Godavari_Figures → Krishna_NDWI_Change_1990-2025.tif")

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

krishna_ndwi_path = '/content/drive/MyDrive/Godavari_Figures/Krishna_NDWI_Change_1990-2025.tif'

with rasterio.open(krishna_ndwi_path) as src:
    data = src.read(1, masked=True)
    valid = data[~data.mask]

    print("Krishna NDWI Change (1990–2025) statistics:")
    print("Min (strong loss/erosion):", np.nanmin(valid))
    print("Max (strong gain/accretion):", np.nanmax(valid))
    print("Mean:", np.nanmean(valid))
    print("Std:", np.nanstd(valid))

    fig, ax = plt.subplots(figsize=(10, 8))
    im = ax.imshow(data, extent=[src.bounds.left, src.bounds.right,
                                 src.bounds.bottom, src.bounds.top],
                   cmap='RdBu_r', vmin=-0.5, vmax=0.5, interpolation='nearest')
    ax.set_title("Krishna NDWI Change (1990–2025)")
    ax.set_xlabel('Longitude (°E)')
    ax.set_ylabel('Latitude (°N)')
    plt.colorbar(im, ax=ax, label='NDWI Change (older - recent)')
    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/Godavari_Figures/Krishna_NDWI_preview.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

cauvery_vlm_path = '/content/drive/MyDrive/Godavari_Figures/cauvery_vlm.tif'  # adjust name if needed

with rasterio.open(cauvery_vlm_path) as src:
    data = src.read(1, masked=True)
    valid = data[~data.mask]

    print("Cauvery 1 km Subsidence stats (mm/yr):")
    print("Min:", np.nanmin(valid))
    print("Max:", np.nanmax(valid))
    print("Mean:", np.nanmean(valid))

    fig, ax = plt.subplots(figsize=(8, 6))
    im = ax.imshow(data, extent=[src.bounds.left, src.bounds.right,
                                 src.bounds.bottom, src.bounds.top],
                   cmap='RdYlBu_r', vmin=-5, vmax=1, interpolation='nearest')
    ax.set_title("Cauvery Subsidence Placeholder (1 km from Zenodo)")
    plt.colorbar(im, ax=ax, label='Vertical Land Motion (mm/yr)')
    plt.show()

In [ ]:
import ee

# Re-initialize if needed
PROJECT_ID = 'project-b75af78d-ff46-4d69-ab3'
ee.Initialize(project=PROJECT_ID)

# Cauvery Delta AOI (covers Pichavaram mangroves, lower distributaries, Nagapattinam coast)
cauvery_aoi = ee.Geometry.Rectangle([79.4, 10.8, 80.0, 11.6])

def add_ndwi(image):
    ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')
    return image.addBands(ndwi)

# Older: 1990–2010 Landsat 5/7 median
ls_old_c = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')) \
    .filterBounds(cauvery_aoi) \
    .filterDate('1990-01-01', '2010-12-31') \
    .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
    .map(add_ndwi) \
    .select('NDWI') \
    .median().clip(cauvery_aoi)

# Recent: 2015–2025 Landsat 8/9 median
ls_recent_c = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .merge(ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')) \
    .filterBounds(cauvery_aoi) \
    .filterDate('2015-01-01', '2025-12-31') \
    .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
    .map(add_ndwi) \
    .select('NDWI') \
    .median().clip(cauvery_aoi)

# Change: older - recent (negative = loss/erosion)
ndwi_change_cauvery = ls_old_c.subtract(ls_recent_c).rename('NDWI_Change')

# Export
task_cauvery = ee.batch.Export.image.toDrive(
    image=ndwi_change_cauvery,
    description='Cauvery_NDWI_Change_1990-2025',
    folder='Godavari_Figures',
    scale=30,
    region=cauvery_aoi,
    maxPixels=1e10
)
task_cauvery.start()

print("Cauvery NDWI export started.")
print("Task ID:", task_cauvery.id)
print("Monitor: https://code.earthengine.google.com/tasks")
print("Output: Drive → Godavari_Figures → Cauvery_NDWI_Change_1990-2025.tif")

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

cauvery_ndwi_path = '/content/drive/MyDrive/Godavari_Figures/Cauvery_NDWI_Change_1990-2025.tif'

with rasterio.open(cauvery_ndwi_path) as src:
    data = src.read(1, masked=True)
    valid = data[~data.mask]

    print("Cauvery NDWI Change (1990–2025) statistics:")
    print("Min (strong loss/erosion):", np.nanmin(valid))
    print("Max (strong gain/accretion):", np.nanmax(valid))
    print("Mean:", np.nanmean(valid))
    print("Std:", np.nanstd(valid))

    fig, ax = plt.subplots(figsize=(10, 8))
    im = ax.imshow(data, extent=[src.bounds.left, src.bounds.right,
                                 src.bounds.bottom, src.bounds.top],
                   cmap='RdBu_r', vmin=-0.5, vmax=0.5, interpolation='nearest')
    ax.set_title("Cauvery NDWI Change (1990–2025)")
    ax.set_xlabel('Longitude (°E)')
    ax.set_ylabel('Latitude (°N)')
    plt.colorbar(im, ax=ax, label='NDWI Change (older - recent)')
    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/Godavari_Figures/Cauvery_NDWI_preview.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
import rasterio
from rasterio.warp import reproject, Resampling
import numpy as np
import matplotlib.pyplot as plt

drive_folder = '/content/drive/MyDrive/Godavari_Figures/'

cau_vlm_path = drive_folder + 'cauvery_vlm.tif'
cau_ndwi_path = drive_folder + 'Cauvery_NDWI_Change_1990-2025.tif'
output_cau_vuln = drive_folder + 'Cauvery_Vulnerability_Hotspots.tif'

# 1. Load subsidence (target grid)
with rasterio.open(cau_vlm_path) as vlm_src:
    vlm_cau = vlm_src.read(1, masked=True)
    vlm_meta = vlm_src.meta.copy()
    vlm_transform = vlm_src.transform
    target_shape = vlm_cau.shape

# 2. Reproject NDWI change to exactly match subsidence grid
with rasterio.open(cau_ndwi_path) as ndwi_src:
    ndwi_reproj = np.empty(target_shape, dtype=np.float32)
    reproject(
        source=rasterio.band(ndwi_src, 1),
        destination=ndwi_reproj,
        src_transform=ndwi_src.transform,
        src_crs=ndwi_src.crs,
        dst_transform=vlm_transform,
        dst_crs=vlm_src.crs,
        resampling=Resampling.average  # smooth downsampling from 30 m to 1 km
    )
    ndwi_cau = np.ma.masked_invalid(ndwi_reproj)

# 3. Threshold-based vulnerability (now same shape)
vuln_cau = np.zeros_like(vlm_cau, dtype=np.float32)
vuln_cau[(vlm_cau < -1) & (ndwi_cau < -0.1)] = 2.0   # high
vuln_cau[(vlm_cau < -1) | (ndwi_cau < -0.1)] = 1.0   # medium

# Save as GeoTIFF
with rasterio.open(output_cau_vuln, 'w', **vlm_meta) as dst:
    dst.write(vuln_cau.astype(np.float32), 1)

print(f"Cauvery vulnerability hotspots saved: {output_cau_vuln}")

# Quick preview
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(vuln_cau, extent=[vlm_src.bounds.left, vlm_src.bounds.right,
                                 vlm_src.bounds.bottom, vlm_src.bounds.top],
               cmap='OrRd', vmin=0, vmax=2, interpolation='nearest')
ax.set_title("Cauvery Vulnerability Hotspots (placeholder)")
plt.colorbar(im, ax=ax, label='Vulnerability Level (0=low, 1=medium, 2=high)')
plt.show()

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

cau_vuln_path = '/content/drive/MyDrive/Godavari_Figures/Cauvery_Vulnerability_Hotspots.tif'

with rasterio.open(cau_vuln_path) as src:
    data = src.read(1, masked=True)
    valid = data[~data.mask]

    print("Cauvery Vulnerability Hotspots stats (0=low, 1=medium, 2=high):")
    print("Min:", np.nanmin(valid))
    print("Max:", np.nanmax(valid))
    print("Mean:", np.nanmean(valid))
    print("Percentage high (2):", (np.count_nonzero(valid == 2) / valid.size) * 100 if valid.size > 0 else 0, "%")

    fig, ax = plt.subplots(figsize=(8, 6))
    im = ax.imshow(data, extent=[src.bounds.left, src.bounds.right,
                                 src.bounds.bottom, src.bounds.top],
                   cmap='OrRd', vmin=0, vmax=2, interpolation='nearest')
    ax.set_title("Cauvery Vulnerability Hotspots (threshold-based, placeholder)")
    ax.set_xlabel('Longitude (°E)')
    ax.set_ylabel('Latitude (°N)')
    plt.colorbar(im, ax=ax, label='Vulnerability Level')
    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/Godavari_Figures/Cauvery_Vuln_preview.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from affine import Affine

drive_folder = '/content/drive/MyDrive/Godavari_Figures/'

# Godavari paths
god_vlm  = drive_folder + 'Godavari_VLM_highres.tif'
god_ndwi = drive_folder + 'Godavari_NDWI_Change_1990-2025.tif'
god_vuln = drive_folder + 'Godavari_Vulnerability_Hotspots_Improved.tif'

# Cauvery paths
cau_vlm  = drive_folder + 'cauvery_vlm.tif'
cau_ndwi = drive_folder + 'Cauvery_NDWI_Change_1990-2025.tif'
cau_vuln = drive_folder + 'Cauvery_Vulnerability_Hotspots.tif'  # newly generated

fig, axs = plt.subplots(3, 2, figsize=(14, 18), dpi=300)
fig.suptitle('Supplementary Figure S1 | Comparative subsidence, NDWI change, and vulnerability in Godavari (left) and Cauvery (right) deltas', fontsize=16, y=0.98)

# Godavari subsidence
with rasterio.open(god_vlm) as src:
    data = src.read(1, masked=True)
    im = axs[0,0].imshow(data, extent=[src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top],
                         cmap='RdYlBu_r', vmin=-50, vmax=10, interpolation='nearest')
axs[0,0].set_title('(a) Godavari subsidence (~75 m)')
axs[0,0].set_xlabel('Longitude (°E)')
axs[0,0].set_ylabel('Latitude (°N)')
fig.colorbar(im, ax=axs[0,0], orientation='vertical', fraction=0.046, pad=0.04).set_label('mm/yr')

# Cauvery subsidence (1 km)
with rasterio.open(cau_vlm) as src:
    data = src.read(1, masked=True)
    im = axs[0,1].imshow(data, extent=[src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top],
                         cmap='RdYlBu_r', vmin=-5, vmax=1, interpolation='nearest')
axs[0,1].set_title('(b) Cauvery subsidence (1 km placeholder)')
axs[0,1].set_xlabel('Longitude (°E)')
axs[0,1].set_ylabel('Latitude (°N)')

# Godavari NDWI change
with rasterio.open(god_ndwi) as src:
    data = src.read(1, masked=True)
    im = axs[1,0].imshow(data, extent=[src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top],
                         cmap='RdBu_r', vmin=-0.5, vmax=0.5, interpolation='nearest')
axs[1,0].set_title('(c) Godavari NDWI change (1990–2025)')
axs[1,0].set_xlabel('Longitude (°E)')
axs[1,0].set_ylabel('Latitude (°N)')

# Cauvery NDWI change
with rasterio.open(cau_ndwi) as src:
    data = src.read(1, masked=True)
    im = axs[1,1].imshow(data, extent=[src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top],
                         cmap='RdBu_r', vmin=-0.5, vmax=0.5, interpolation='nearest')
axs[1,1].set_title('(d) Cauvery NDWI change (1990–2025)')
axs[1,1].set_xlabel('Longitude (°E)')
axs[1,1].set_ylabel('Latitude (°N)')

# Godavari vulnerability
with rasterio.open(god_vuln) as src:
    data = src.read(1, masked=True)
    im = axs[2,0].imshow(data, extent=[src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top],
                         cmap='OrRd', vmin=0, vmax=2, interpolation='nearest')
axs[2,0].set_title('(e) Godavari vulnerability hotspots')
axs[2,0].set_xlabel('Longitude (°E)')
axs[2,0].set_ylabel('Latitude (°N)')

# Cauvery vulnerability (already generated)
with rasterio.open(cau_vuln) as src:
    data = src.read(1, masked=True)
    im = axs[2,1].imshow(data, extent=[src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top],
                         cmap='OrRd', vmin=0, vmax=2, interpolation='nearest')
axs[2,1].set_title('(f) Cauvery vulnerability hotspots (placeholder)')
axs[2,1].set_xlabel('Longitude (°E)')
axs[2,1].set_ylabel('Latitude (°N)')

# Legend
legend_elements = [
    Patch(facecolor='darkred', label='High vulnerability (2)'),
    Patch(facecolor='orange', label='Medium (1)'),
    Patch(facecolor='lightyellow', label='Low (0)')
]
fig.legend(handles=legend_elements, loc='lower center', ncol=3, bbox_to_anchor=(0.5, -0.02), fontsize=10)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(drive_folder + 'Supp_Fig_S1_Godavari_vs_Cauvery.svg', format='svg', bbox_inches='tight', dpi=300)
print("Supplementary Figure S1 saved:", drive_folder + 'Supp_Fig_S1_Godavari_vs_Cauvery.svg')
plt.show()

In [ ]:
import rasterio
import numpy as np

drive_folder = '/content/drive/MyDrive/Godavari_Figures/'

# Load final Godavari layers
with rasterio.open(drive_folder + 'Godavari_VLM_highres.tif') as src:
    vlm = src.read(1, masked=True)
    pixel_area_km2 = (src.res[0] * src.res[1]) / 1e6  # ~75 m pixels → km²

with rasterio.open(drive_folder + 'Godavari_NDWI_Change_1990-2025.tif') as src:
    ndwi_change = src.read(1, masked=True)

with rasterio.open(drive_folder + 'Godavari_Vulnerability_Hotspots_Improved.tif') as src:
    vuln = src.read(1, masked=True)

# Total analysed area
total_valid_pixels = np.count_nonzero(~vlm.mask)
total_area_km2 = total_valid_pixels * pixel_area_km2

# Vulnerability areas
high_vuln_pixels = np.count_nonzero(vuln == 2)
medium_vuln_pixels = np.count_nonzero(vuln == 1)
high_vuln_area_km2 = high_vuln_pixels * pixel_area_km2
medium_vuln_area_km2 = medium_vuln_pixels * pixel_area_km2

# Mangrove fringe retreat (NDWI change < -0.1 as proxy)
retreat_pixels = np.count_nonzero(ndwi_change < -0.1)
retreat_area_km2 = retreat_pixels * pixel_area_km2

print("=== Quantitative Stats – Godavari Delta Front ===")
print(f"Total analysed area: {total_area_km2:.1f} km²")
print(f"High vulnerability hotspots (level 2): {high_vuln_area_km2:.1f} km² ({high_vuln_pixels/total_valid_pixels*100:.1f} %)")
print(f"Medium vulnerability areas (level 1): {medium_vuln_area_km2:.1f} km²")
print(f"Coastal fringe in retreat (NDWI change < -0.1): {retreat_area_km2:.1f} km² ({retreat_pixels/total_valid_pixels*100:.1f} % of total)")

In [ ]:
import rasterio
import numpy as np

drive_folder = '/content/drive/MyDrive/Godavari_Figures/'

# Approximate meters per degree at 16°N latitude
deg_to_m_lon = 111320 * np.cos(np.deg2rad(16))  # longitude scaling
deg_to_m_lat = 111320  # latitude scaling (almost constant)

with rasterio.open(drive_folder + 'Godavari_VLM_highres.tif') as src:
    vlm = src.read(1, masked=True)
    res_lon_deg = src.res[0]  # degrees
    res_lat_deg = src.res[1]
    pixel_area_m2 = (res_lon_deg * deg_to_m_lon) * (res_lat_deg * deg_to_m_lat)
    pixel_area_km2 = pixel_area_m2 / 1e6

with rasterio.open(drive_folder + 'Godavari_NDWI_Change_1990-2025.tif') as src:
    ndwi_change = src.read(1, masked=True)

with rasterio.open(drive_folder + 'Godavari_Vulnerability_Hotspots_Improved.tif') as src:
    vuln = src.read(1, masked=True)

# Calculations
total_valid_pixels = np.count_nonzero(~vlm.mask)
total_area_km2 = total_valid_pixels * pixel_area_km2

high_vuln_pixels = np.count_nonzero(vuln == 2)
medium_vuln_pixels = np.count_nonzero(vuln == 1)
high_vuln_area_km2 = high_vuln_pixels * pixel_area_km2
medium_vuln_area_km2 = medium_vuln_pixels * pixel_area_km2

retreat_pixels = np.count_nonzero(ndwi_change < -0.1)
retreat_area_km2 = retreat_pixels * pixel_area_km2

print("=== Quantitative Stats – Godavari Delta Front (corrected) ===")
print(f"Pixel size (approx): {pixel_area_km2:.6f} km²")
print(f"Total analysed area: {total_area_km2:.1f} km²")
print(f"High vulnerability hotspots (level 2): {high_vuln_area_km2:.1f} km² ({high_vuln_pixels / total_valid_pixels * 100:.1f} %)")
print(f"Medium vulnerability areas (level 1): {medium_vuln_area_km2:.1f} km² ({medium_vuln_pixels / total_valid_pixels * 100:.1f} %)")
print(f"Coastal fringe in retreat (NDWI change < -0.1): {retreat_area_km2:.1f} km² ({retreat_pixels / total_valid_pixels * 100:.1f} %)")

In [ ]:
import ee

# Re-initialize if needed
PROJECT_ID = 'project-b75af78d-ff46-4d69-ab3'
ee.Initialize(project=PROJECT_ID)

# Godavari AOI (same as before)
god_aoi = ee.Geometry.Rectangle([81.4, 15.9, 83.1, 17.6])

def add_ndwi(image):
    ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')
    return image.addBands(ndwi)

# Period 1: 2000–2010 (Landsat 5/7)
ls_2000_2010 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')) \
    .filterBounds(god_aoi) \
    .filterDate('2000-01-01', '2010-12-31') \
    .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
    .map(add_ndwi) \
    .select('NDWI') \
    .median().clip(god_aoi)

# Period 2: 2015–2025 (Landsat 8/9)
ls_2015_2025 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .merge(ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')) \
    .filterBounds(god_aoi) \
    .filterDate('2015-01-01', '2025-12-31') \
    .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
    .map(add_ndwi) \
    .select('NDWI') \
    .median().clip(god_aoi)

# Export both periods
task_early = ee.batch.Export.image.toDrive(
    image=ls_2000_2010,
    description='Godavari_NDWI_2000-2010',
    folder='Godavari_Figures',
    scale=30,
    region=god_aoi,
    maxPixels=1e10
)
task_early.start()

task_late = ee.batch.Export.image.toDrive(
    image=ls_2015_2025,
    description='Godavari_NDWI_2015-2025',
    folder='Godavari_Figures',
    scale=30,
    region=god_aoi,
    maxPixels=1e10
)
task_late.start()

print("Temporal NDWI exports started.")
print("Tasks: Godavari_NDWI_2000-2010.tif and Godavari_NDWI_2015-2025.tif")
print("Monitor: https://code.earthengine.google.com/tasks")

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

drive_folder = '/content/drive/MyDrive/Godavari_Figures/'

# Early period (2000–2010)
early_path = drive_folder + 'Godavari_NDWI_2000-2010.tif'
with rasterio.open(early_path) as src:
    early_ndwi = src.read(1, masked=True)
    early_valid = early_ndwi[~early_ndwi.mask]

    print("=== Godavari NDWI 2000–2010 ===")
    print("Min:", np.nanmin(early_valid))
    print("Max:", np.nanmax(early_valid))
    print("Mean:", np.nanmean(early_valid))

# Recent period (2015–2025)
late_path = drive_folder + 'Godavari_NDWI_2015-2025.tif'
with rasterio.open(late_path) as src:
    late_ndwi = src.read(1, masked=True)
    late_valid = late_ndwi[~late_ndwi.mask]

    print("\n=== Godavari NDWI 2015–2025 ===")
    print("Min:", np.nanmin(late_valid))
    print("Max:", np.nanmax(late_valid))
    print("Mean:", np.nanmean(late_valid))

# Simple difference preview (early - late; negative = NDWI loss over time)
diff = early_ndwi - late_ndwi
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(diff, extent=[src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top],
               cmap='RdBu_r', vmin=-0.5, vmax=0.5, interpolation='nearest')
ax.set_title("NDWI Difference (2000–2010 minus 2015–2025)")
ax.set_xlabel('Longitude (°E)')
ax.set_ylabel('Latitude (°N)')
plt.colorbar(im, ax=ax, label='NDWI Change')
plt.tight_layout()
plt.savefig(drive_folder + 'Godavari_NDWI_Temporal_Diff_preview.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import rasterio
import numpy as np

drive_folder = '/content/drive/MyDrive/Godavari_Figures/'

# Load subsidence (fixed high-res layer)
with rasterio.open(drive_folder + 'Godavari_VLM_highres.tif') as src:
    vlm = src.read(1, masked=True)
    vlm_meta = src.meta.copy()
    pixel_area_km2 = (src.res[0] * 111320 * np.cos(np.deg2rad(16))) * (src.res[1] * 111320) / 1e6  # accurate km²

# Load temporal NDWI
with rasterio.open(drive_folder + 'Godavari_NDWI_2000-2010.tif') as src:
    ndwi_early = src.read(1, masked=True)

with rasterio.open(drive_folder + 'Godavari_NDWI_2015-2025.tif') as src:
    ndwi_late = src.read(1, masked=True)

# Vulnerability for each period (same thresholds as before)
def compute_vuln(ndwi, vlm):
    vuln = np.zeros_like(vlm, dtype=np.float32)
    vuln[(vlm < -5) & (ndwi < -0.1)] = 2.0   # high
    vuln[(vlm < -5) | (ndwi < -0.1)] = 1.0   # medium
    return vuln

vuln_early = compute_vuln(ndwi_early, vlm)
vuln_late  = compute_vuln(ndwi_late, vlm)

# Area calculations
total_pixels = np.count_nonzero(~vlm.mask)
total_area_km2 = total_pixels * pixel_area_km2

def vuln_area(v):
    high = np.count_nonzero(v == 2) * pixel_area_km2
    med  = np.count_nonzero(v == 1) * pixel_area_km2
    return high, med

high_early, med_early = vuln_area(vuln_early)
high_late,  med_late  = vuln_area(vuln_late)

print("=== Temporal Vulnerability Stats – Godavari ===")
print(f"Total analysed area: {total_area_km2:.1f} km²")
print("\n2000–2010:")
print(f"High vuln (2): {high_early:.1f} km² ({high_early/total_area_km2*100:.1f} %)")
print(f"Medium vuln (1): {med_early:.1f} km²")
print("\n2015–2025:")
print(f"High vuln (2): {high_late:.1f} km² ({high_late/total_area_km2*100:.1f} %)")
print(f"Medium vuln (1): {med_late:.1f} km²")
print(f"\nChange in high vuln area: {high_late - high_early:.1f} km² ({(high_late - high_early)/high_early*100 if high_early > 0 else 'N/A'} %)")

In [ ]:
import rasterio
from rasterio.warp import reproject, Resampling
import numpy as np

drive_folder = '/content/drive/MyDrive/Godavari_Figures/'

# 1. Load subsidence (target grid)
with rasterio.open(drive_folder + 'Godavari_VLM_highres.tif') as src:
    vlm = src.read(1, masked=True)
    vlm_meta = src.meta.copy()
    vlm_transform = src.transform
    target_shape = vlm.shape
    pixel_area_km2 = (src.res[0] * 111320 * np.cos(np.deg2rad(16))) * (src.res[1] * 111320) / 1e6

# 2. Load & reproject early NDWI (2000–2010)
with rasterio.open(drive_folder + 'Godavari_NDWI_2000-2010.tif') as src:
    ndwi_early_reproj = np.empty(target_shape, dtype=np.float32)
    reproject(
        source=rasterio.band(src, 1),
        destination=ndwi_early_reproj,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=vlm_transform,
        dst_crs=vlm_src.crs,
        resampling=Resampling.average
    )
    ndwi_early = np.ma.masked_invalid(ndwi_early_reproj)

# 3. Load & reproject late NDWI (2015–2025)
with rasterio.open(drive_folder + 'Godavari_NDWI_2015-2025.tif') as src:
    ndwi_late_reproj = np.empty(target_shape, dtype=np.float32)
    reproject(
        source=rasterio.band(src, 1),
        destination=ndwi_late_reproj,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=vlm_transform,
        dst_crs=vlm_src.crs,
        resampling=Resampling.average
    )
    ndwi_late = np.ma.masked_invalid(ndwi_late_reproj)

# 4. Compute vulnerability for each period
def compute_vuln(ndwi, vlm):
    vuln = np.zeros_like(vlm, dtype=np.float32)
    vuln[(vlm < -5) & (ndwi < -0.1)] = 2.0   # high
    vuln[(vlm < -5) | (ndwi < -0.1)] = 1.0   # medium
    return vuln

vuln_early = compute_vuln(ndwi_early, vlm)
vuln_late  = compute_vuln(ndwi_late, vlm)

# 5. Area calculations
total_pixels = np.count_nonzero(~vlm.mask)
total_area_km2 = total_pixels * pixel_area_km2

def vuln_area(v):
    high = np.count_nonzero(v == 2) * pixel_area_km2
    med  = np.count_nonzero(v == 1) * pixel_area_km2
    return high, med

high_early, med_early = vuln_area(vuln_early)
high_late,  med_late  = vuln_area(vuln_late)

print("=== Temporal Vulnerability Stats – Godavari ===")
print(f"Total analysed area: {total_area_km2:.1f} km²")
print("\n2000–2010:")
print(f"High vuln (2): {high_early:.1f} km² ({high_early/total_area_km2*100:.1f} %)")
print(f"Medium vuln (1): {med_early:.1f} km²")
print("\n2015–2025:")
print(f"High vuln (2): {high_late:.1f} km² ({high_late/total_area_km2*100:.1f} %)")
print(f"Medium vuln (1): {med_late:.1f} km²")
print(f"\nChange in high vuln area (2015–2025 minus 2000–2010): {high_late - high_early:.1f} km²")
print(f"Change %: {((high_late - high_early)/high_early*100) if high_early > 0 else 'N/A'} %")